In [ ]:
# Imports 

import pandas as pd
import acquire as a 

## Bring in our dataframe

In [1]:
df = a.acquire_zillow_data()
df.head(1)

NameError: name 'a' is not defined

In [43]:
import pandas as pd
import os 

def add_county_state(df): 
  
    if os.path.exists('state_and_county_fips_master.csv') == True:
        fips = pd.read_csv('state_and_county_fips_master.csv')
    else: 
        url = 'https://raw.githubusercontent.com/kjhealy/fips-codes/master/state_and_county_fips_master.csv'
        fips = pd.read_csv(url)
        fips.to_csv('state_and_county_fips_master.csv')
    fips['county_state']= fips['name'] + ', ' + fips['state']
    df = pd.merge(df,fips,on='fips')
    df = df.drop(columns = ['name','state', 'fips'])
    
    return df
    
def prepare(df):
   
    """
    Prepares the given DataFrame by performing data cleaning and preprocessing steps.
    
    Args:
        df (DataFrame): Input DataFrame containing Zillow data
        
    Returns:
        DataFrame: Cleaned and preprocessed DataFrame
        
    """
    # Drop unnecessary columns and rows with missing values
    df = df.drop(columns='Unnamed: 0',inplace=True)
    df = df.dropna()
    
    # Rename columns for better readability
    cols_name = ['bedrooms', 'bathrooms', 'calculated_finished_squarefeet',
           'tax_valuedollar_cnt', 'year_built', 'tax_amount', 'fips']
    df = df.set_axis(cols_name, axis=1)
    df['year_built'] = df['year_built'].astype(int)
    
    # Handle Outliers:
    # The general rule for outliers are:
    ## Upper bound: Q3 + 1.5*IQR
    ## Lower bound: Q1 - 1.5*IQR
    # Bounds are manually adjusted for each feature
    df = df[df.bedrooms <= 6]
    df = df[df.bedrooms >= 1]

    df = df[df.bathrooms <= 6.5]
    df = df[df.bathrooms >= 0.5]

    df = df[df.calculated_finished_squarefeet <= 4800]
    df = df[df.calculated_finished_squarefeet >= 500]
    
    # Add county and state columns to the data
    df = add_county_state(df)
    
    # Get dummy values for object columns
    df = pd.get_dummies(df, columns=['county_state'])
    
    # Clean up the column names for easier processing
    df.rename(columns={
        'county_state_Los Angeles County, CA': 'la_county_ca',
        'county_state_Orange County, CA': 'orange_county_ca',
        'county_state_Ventura County, CA': 'ventura_county_ca'
    }, inplace=True)
    
    cols = list(df.columns)
    cols.append(cols.pop(cols.index('tax_valuedollar_cnt')))
    df = df[cols]

    # Return the cleaned and preprocessed data
    return df


In [44]:
df

,Unnamed: 0,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
...,...,...,...,...,...,...,...,...
2152858,2152858,4.0,3.0,2262.0,960756.0,2015.0,13494.52,6059.0
2152859,2152859,4.0,4.5,3127.0,536061.0,2014.0,6244.16,6059.0
2152860,2152860,0.0,0.0,NaN,208057.0,NaN,5783.88,6059.0
2152861,2152861,3.0,2.5,1974.0,424353.0,2015.0,5302.70,6059.0


In [45]:
"""
Prepares the given DataFrame by performing data cleaning and preprocessing steps.

Args:
    df (DataFrame): Input DataFrame containing Zillow data

Returns:
    DataFrame: Cleaned and preprocessed DataFrame

"""
# Drop unnecessary columns and rows with missing values
df.drop(columns='Unnamed: 0',inplace=True)
df = df.dropna(ignore_index=True)

# Rename columns for better readability
cols_name = ['bedrooms', 'bathrooms', 'calculated_finished_squarefeet',
       'tax_valuedollar_cnt', 'year_built', 'tax_amount', 'fips']
df = df.set_axis(cols_name, axis=1)
df['year_built'] = df['year_built'].astype(int)

# Handle Outliers:
# The general rule for outliers are:
## Upper bound: Q3 + 1.5*IQR
## Lower bound: Q1 - 1.5*IQR
# Bounds are manually adjusted for each feature

df = df[df.bedrooms <= 6]
df = df[df.bedrooms >= 1]

df = df[df.bathrooms <= 6.5]
df = df[df.bathrooms >= 0.5]

df = df[df.calculated_finished_squarefeet <= 4800]
df = df[df.calculated_finished_squarefeet >= 500]

# Add county and state columns to the data
df = add_county_state(df)

# Get dummy values for object columns
df = pd.get_dummies(df, columns=['county_state'])

# Clean up the column names for easier processing
df.rename(columns={
    'county_state_Los Angeles County, CA': 'la_county_ca',
    'county_state_Orange County, CA': 'orange_county_ca',
    'county_state_Ventura County, CA': 'ventura_county_ca'
}, inplace=True)

cols = list(df.columns)
cols.append(cols.pop(cols.index('tax_valuedollar_cnt')))
df = df[cols]

In [46]:
df.head()

,bedrooms,bathrooms,calculated_finished_squarefeet,year_built,tax_amount,Unnamed: 0,la_county_ca,orange_county_ca,ventura_county_ca,tax_valuedollar_cnt
0,4.0,2.0,3633.0,2005,6941.39,210,True,False,False,296425.0
1,3.0,4.0,1620.0,2011,10244.94,210,True,False,False,847770.0
2,3.0,2.0,2077.0,1926,7924.68,210,True,False,False,646760.0
3,3.0,1.0,1244.0,1950,2532.88,210,True,False,False,169471.0
4,3.0,2.0,1300.0,1950,3110.99,210,True,False,False,233266.0


In [47]:
df.drop(columns='Unnamed: 0',inplace=True)

In [58]:
df.head(30)
counties = []
for cols in df.columns:
    if 'county' in cols:
        counties.append(cols)
    else:
        pass
        
for cols in counties:
    df[f'{cols}'] = df[f'{cols}'].astype(int)

In [59]:
df

,bedrooms,bathrooms,calculated_finished_squarefeet,year_built,tax_amount,la_county_ca,orange_county_ca,ventura_county_ca,tax_valuedollar_cnt
0,4.0,2.0,3633.0,2005,6941.39,1,0,0,296425.0
1,3.0,4.0,1620.0,2011,10244.94,1,0,0,847770.0
2,3.0,2.0,2077.0,1926,7924.68,1,0,0,646760.0
3,3.0,1.0,1244.0,1950,2532.88,1,0,0,169471.0
4,3.0,2.0,1300.0,1950,3110.99,1,0,0,233266.0
...,...,...,...,...,...,...,...,...,...
2096774,2.0,1.0,812.0,1951,6010.56,0,0,1,566923.0
2096775,4.0,2.0,1891.0,1976,4628.30,0,0,1,395704.0
2096776,5.0,4.0,3611.0,2015,12785.44,0,0,1,1214890.0
2096777,2.0,1.0,710.0,1940,2040.78,0,0,1,175319.0
